In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Demo").getOrCreate()
spark

23/04/04 20:36:52 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/04/04 20:36:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 20:36:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 20:36:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/04 20:36:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/04 20:36:53 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
empDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/employees.csv")
empDf.show(1)

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
only showing top 1 row



In [5]:
deptDf = spark.read.option("header",True).option("infrechema",True).csv("../../Spark-main/departments.csv")
deptDf.show(1)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
+-------------+---------------+----------+-----------+
only showing top 1 row



In [6]:
from pyspark.sql.functions import *

In [9]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
#The Value is in Bytes

#Note : We can diasable the Braodcast Join
#spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1) #Disable

In [8]:
empDf.join(broadcast(deptDf), empDf.DEPARTMENT_ID == deptDf.DEPARTMENT_ID, "inner").select(empDf.EMPLOYEE_ID, empDf.DEPARTMENT_ID, deptDf.DEPARTMENT_NAME).show(100)

+-----------+-------------+----------------+
|EMPLOYEE_ID|DEPARTMENT_ID| DEPARTMENT_NAME|
+-----------+-------------+----------------+
|        198|           50|        Shipping|
|        199|           50|        Shipping|
|        200|           10|  Administration|
|        201|           20|       Marketing|
|        202|           20|       Marketing|
|        203|           40| Human Resources|
|        204|           70|Public Relations|
|        205|          110|      Accounting|
|        206|          110|      Accounting|
|        100|           90|       Executive|
|        101|           90|       Executive|
|        102|           90|       Executive|
|        103|           60|              IT|
|        104|           60|              IT|
|        105|           60|              IT|
|        106|           60|              IT|
|        107|           60|              IT|
|        108|          100|         Finance|
|        109|          100|         Finance|
|        1